In [1]:
import psycopg2
import torch
import pickle
import sqlite3
import pandas as pd
import numpy as np

In [2]:
def connect():
    return psycopg2.connect(database="music_embeddings",
                        host="localhost",
                        user="maxj",
                        password="",
                        port="5432")


```
>>> import psycopg2

# Connect to an existing database
>>> conn = psycopg2.connect("dbname=test user=postgres")

# Open a cursor to perform database operations
>>> cur = conn.cursor()

# Execute a command: this creates a new table
>>> cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")

# Pass data to fill a query placeholders and let Psycopg perform
# the correct conversion (no more SQL injections!)
>>> cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)",
...      (100, "abc'def"))

# Query the database and obtain data as Python objects
>>> cur.execute("SELECT * FROM test;")
>>> cur.fetchone()
(1, 100, "abc'def")

# Make the changes to the database persistent
>>> conn.commit()

# Close communication with the database
>>> cur.close()
>>> conn.close()
```

In [3]:
def reset_tracks():
    conn = connect()
    c = conn.cursor()
    c.execute("drop table if exists tracks;")
    c.execute("""
    CREATE TABLE tracks (
      id BIGINT PRIMARY KEY,
      path VARCHAR(255) NOT NULL UNIQUE,
      artist TEXT,
      title TEXT NOT NULL,
      bpm INT,
      key VARCHAR(7),
      embedding_musicgen_medium vector(1536)
    );
    """)
    conn.commit()
    c.close()
    conn.close()

reset_tracks()

In [4]:
try:
    with open('embeddings.pickle', 'rb') as f:
        embs = pickle.load(f)
        print('cached embeddings', len(embs))
except FileNotFoundError:
    print('empty embeddings')
    embs = {}

cached embeddings 1249


In [5]:
q = """
select 
  djmdContent.ID as id, 
  djmdContent.FolderPath as path, 
  djmdArtist.Name as artist, 
  djmdContent.Title as title, 
  djmdContent.BPM as bpm, 
  djmdKey.ScaleName as key 
from 
  djmdContent 
  left join djmdArtist on djmdArtist.ID = djmdContent.ArtistID 
  left join djmdKey on djmdKey.ID = djmdContent.KeyID 
where 
  djmdContent.rb_local_deleted = 0;
""".strip()
con = sqlite3.connect("plaintext.db")
df_tracks = pd.read_sql_query(q, con)
con.close()
df_tracks

,id,path,artist,title,bpm,key
0,178582527,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...,None,NOISE,0,None
1,35341928,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...,None,SINEWAVE,0,None
2,196758694,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...,None,SIREN,0,None
3,96986769,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...,None,HORN,0,None
4,78147589,/Users/maxj/Music/PioneerDJ/Demo Tracks/Demo T...,Loopmasters,Demo Track 1 [-12],12800,None
...,...,...,...,...,...,...
1255,233763711,/Users/maxj/Music/Rips/Spotify/AIFF/Nikitch - ...,"Nikitch, Mart One",Wonda [-11],7250,10A
1256,238880670,/Users/maxj/Music/Rips/Spotify/AIFF/KAM-BU - E...,KAM-BU,ETON MESS [-8],8000,9A
1257,255506951,/Users/maxj/Music/Rips/Spotify/AIFF/RAPRAVE - ...,"RAPRAVE, Araya, HUNA",Echo - Huna Edit [-11],10200,4A
1258,108746388,/Users/maxj/Music/Rips/Spotify/AIFF/NxxxxxS - ...,"NxxxxxS, CHLOBOCOP",Mosh O'Clock [-8],7250,2A


In [6]:
df_tracks['embedding_musicgen_medium'] = df_tracks['path'].map({p: e.numpy() for p, e in embs.items()})
df_tracks

,id,path,artist,title,bpm,key,embedding_musicgen_medium
0,178582527,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...,None,NOISE,0,None,"[99.74503, -19.897644, 29.04325, -82.68303, 16..."
1,35341928,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...,None,SINEWAVE,0,None,"[17.656895, 46.61774, -34.89798, -137.24483, -..."
2,196758694,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...,None,SIREN,0,None,"[130.40018, -45.352085, -6.155841, -50.946327,..."
3,96986769,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...,None,HORN,0,None,"[259.3045, -149.60623, 36.657417, -130.89412, ..."
4,78147589,/Users/maxj/Music/PioneerDJ/Demo Tracks/Demo T...,Loopmasters,Demo Track 1 [-12],12800,None,"[896.248, -1460.882, 527.2375, -344.6053, 1220..."
...,...,...,...,...,...,...,...
1255,233763711,/Users/maxj/Music/Rips/Spotify/AIFF/Nikitch - ...,"Nikitch, Mart One",Wonda [-11],7250,10A,"[783.24567, -1206.9556, 727.6675, -835.7648, -..."
1256,238880670,/Users/maxj/Music/Rips/Spotify/AIFF/KAM-BU - E...,KAM-BU,ETON MESS [-8],8000,9A,"[-67.24658, -397.87823, 596.7294, 354.0371, 56..."
1257,255506951,/Users/maxj/Music/Rips/Spotify/AIFF/RAPRAVE - ...,"RAPRAVE, Araya, HUNA",Echo - Huna Edit [-11],10200,4A,"[956.83923, -80.69968, -643.6951, 190.1223, 12..."
1258,108746388,/Users/maxj/Music/Rips/Spotify/AIFF/NxxxxxS - ...,"NxxxxxS, CHLOBOCOP",Mosh O'Clock [-8],7250,2A,"[312.26917, 278.28592, -35.87419, 282.3414, 22..."


In [7]:
def populate(df):
    conn = connect()
    c = conn.cursor()
    for index, row in df.iterrows():
        try:
            emb = np.array2string(row["embedding_musicgen_medium"], precision=15, separator=",", threshold=10000)
        except AttributeError:
            emb = None
        c.execute("INSERT INTO tracks VALUES (%s, %s, %s, %s, %s, %s, %s)", (row["id"], row["path"], row["artist"], row["title"], row["bpm"], row["key"], emb))
    conn.commit()
    c.close()
    conn.close()

populate(df_tracks)

In [8]:
def get(path):
    conn = connect()
    c = conn.cursor()
    c.execute('select id, bpm, embedding_musicgen_medium from tracks where path=%s limit 1', (path,))
    id, bpm, emb = c.fetchone()
    c.close()
    conn.close()
    return id, bpm, emb


In [81]:
def sim(id, bpm, emb, limit=5):
    conn = connect()
    c = conn.cursor()
    c.execute(f'''
select 
  id, 
  path, 
  artist, 
  title, 
  key,
  bpm / 100,
  (1 - (embedding_musicgen_medium <=> %s)) as cosine_sim 
from 
  tracks 
where 
  id != %s 
  and embedding_musicgen_medium is not null 
  and (
    bpm >= 0.92 * %s and bpm <= 1.08 * %s 
    or bpm >= 1.84 * %s and bpm <= 2.16 * %s 
    or bpm >= 0.46 * %s and bpm <= 0.54 * %s
  )
order by 
  cosine_sim desc 
limit 
  {limit}''', (emb, id, bpm, bpm, bpm, bpm, bpm, bpm))
    # c.execute('select id, path, artist, title from tracks where id != %s order by embedding_musicgen_medium <=> %s limit 5', (id, emb))
    rows = c.fetchall()
    c.close()
    conn.close()
    return pd.DataFrame.from_records(rows, columns=["id", "path", "artist", "title", "key", "bpm", "cosine_sim"])

In [82]:
sim(*get('/Users/maxj/Music/Edits/Dizzee Rascal - Money Right (mj instrumental intro).aif'))

,id,path,artist,title,key,bpm,cosine_sim
0,17532564,/Users/maxj/Music/Rips/Spotify/AIFF/Dizzee Ras...,"Dizzee Rascal, Skepta",Money Right [-7],7A,75,0.936666
1,116636439,/Users/maxj/Music/Rips/Spotify/AIFF/AllttA - T...,"AllttA, 20syl, Mr. J. Medeiros",The Woods [-10],9A,80,0.824982
2,75066783,/Users/maxj/Music/Rips/Spotify/AIFF/GRiZ - Im ...,GRiZ,I’m Good [-8],7A,80,0.794012
3,27487530,/Users/maxj/Music/Rips/Spotify/AIFF/GRiZ - I'm...,GRiZ,I'm Good [-8],7A,80,0.794012
4,244932714,/Users/maxj/Music/Rips/Spotify/AIFF/Gardna - B...,"Gardna, Eva Lazarus",Bureau De Change [-10],5A,71,0.787927


In [83]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/MALFNKTION - Snake Charmer - 2It0LUgW099w5mBcvKqE14.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,104790003,/Users/maxj/Music/Rips/Spotify/AIFF/MCR-T - My...,"MCR-T, horsegiirL",My Barn My Rules [-12],2A,147,0.766312
1,226726062,/Users/maxj/Music/Rips/Spotify/AIFF/Nikitch - ...,"Nikitch, Mart One",Oussala [-10],2A,75,0.764178
2,45597560,/Users/maxj/Music/Rips/Spotify/AIFF/ill.gates ...,ill.gates,More Tea [-7],12A,70,0.744580
3,149994625,/Users/maxj/Music/Downloads/Soulseek/Interplan...,Interplanetary Criminal,Supreme Level [-9],4A,130,0.742520
4,141800806,/Users/maxj/Music/Rips/Spotify/AIFF/M.A.N.D.Y....,"M.A.N.D.Y., Booka Shade",Body Language - Original Mix [-10],10A,130,0.742024


In [84]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Moka Nola - Reckless - 12 Version - 1Pw33LLNZUQlkqIqBVMKHR.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,51564322,/Users/maxj/Music/Edits/Digital Afrika - Kings...,"Digital Afrika, Cazeaux Oslo",Kings for a Minute [-1st] [-10],9A,124,0.819259
1,82461351,/Users/maxj/Music/Rips/Spotify/AIFF/Athlete Wh...,Athlete Whippet,Nogueira [-11],9A,130,0.808689
2,175907442,/Users/maxj/Music/Rips/Spotify/AIFF/Young Fran...,"Young Franco, Denzel Curry, Pell, Close Counters",Fallin' Apart - Close Counters Remix [-9],9A,123,0.806975
3,237739585,/Users/maxj/Music/Rips/Spotify/AIFF/J-E-T-S - ...,"J-E-T-S, Jesse Boykins III",Pyrite Blue [-9],10A,65,0.789859
4,187361846,/Users/maxj/Music/Downloads/Soulseek/A Tribe C...,"A Tribe Called Quest, Sir Piers, Si Ashton",Bonita Applebum - Sir Piers and Si Ashton's Cu...,10A,122,0.788605


In [85]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Netsky - Tequila Limonada - 5qZRZxbsNd4EQkLZuNpW9u.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,27438909,/Users/maxj/Music/Rips/Spotify/AIFF/CJ Fly - B...,"CJ Fly, Statik Selektah, Kirk Knight",BLOCK PARTY (feat. Kirk Knight) [-7],11A,88,0.846393
1,81108243,/Users/maxj/Music/Rips/Spotify/AIFF/Glass Anim...,Glass Animals,Heat Waves [-6],1A,80,0.778063
2,241797191,/Users/maxj/Music/Rips/Spotify/AIFF/extremely ...,"extremely bad man, Omenihu, Jallal",Bombae Lemonade [-9],2A,81,0.775181
3,224687820,/Users/maxj/Music/Rips/Spotify/AIFF/HashFinger...,HashFinger,Low Down [-10],1A,87,0.772679
4,251940728,/Users/maxj/Music/Rips/Spotify/AIFF/Action Bro...,"Action Bronson, Chance the Rapper",Baby Blue (feat. Chance the Rapper) [-9],12A,84,0.770809


In [86]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Danny Byrd - Planet Earth - 1SJMCmyG3cBVmvGZkTzIK8.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,96946111,/Users/maxj/Music/Rips/Spotify/AIFF/Deekline -...,"Deekline, Ed Solo",King Of The Bong O [-7],8A,175,0.887147
1,141137140,/Users/maxj/Music/Rips/Spotify/AIFF/Danny Byrd...,"Danny Byrd, D Double E",Selecta [-6],5A,174,0.847278
2,196792772,/Users/maxj/Music/Rips/Spotify/AIFF/The Protot...,"The Prototypes, LOWES",Reason [-7],9A,174,0.843526
3,6677030,/Users/maxj/Music/Rips/Spotify/AIFF/Origin8a &...,"Origin8a & Propa, Benny Page",Harmony [-7],8A,174,0.833151
4,101347676,/Users/maxj/Music/Rips/Spotify/AIFF/Chase & St...,"Chase & Status, Clementine Douglas",Mixed Emotions [-8],8A,175,0.827729


In [87]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/DJ Fresh - Gold Dust - SHY FX Re-Edit - 6aOKYH4j0TshPhDkRCGlor.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,17128374,/Users/maxj/Music/Rips/Spotify/AIFF/TroyBoi - ...,"TroyBoi, Amar",AJA AJA [-7],5A,87,0.831888
1,248139986,/Users/maxj/Music/Rips/Spotify/AIFF/Sigma - Li...,"Sigma, Queen Millz",Little Things [-8],9A,174,0.828531
2,223622984,/Users/maxj/Music/Rips/Spotify/AIFF/CLIPZ - No...,"CLIPZ, Nia Archives, Beenie Man, Cristale, Sha...","No Time (feat. Nia Archives, Beenie Man, Crist...",7A,87,0.807320
3,250498795,/Users/maxj/Music/Rips/Spotify/AIFF/Stylust - ...,"Stylust, Gisto, PINEO & LOEB",Bump in the Road (feat. Gisto & Pineo & Loeb) ...,5B,85,0.793976
4,169500296,/Users/maxj/Music/Rips/Spotify/AIFF/Montell209...,"Montell2099, Juelz",Tested [-6],1A,88,0.788763


In [88]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Minnesota - Purple Daze - 1tnhQiG0s0YPQhbFluqVGv.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,233810442,/Users/maxj/Music/Rips/Spotify/AIFF/Big Gigant...,"Big Gigantic, Felly",St. Lucia [-7],8A,82,0.736058
1,117294550,/Users/maxj/Music/Rips/Spotify/AIFF/Soligen - ...,"Soligen, Type2, Break",Can't Go - Break Remix [-8],10A,170,0.727081
2,253622682,/Users/maxj/Music/Rips/Spotify/AIFF/Chinese Ma...,"Chinese Man, Scratch Bandits Crew, Baja Freque...",Be Mine [-8],8A,80,0.726873
3,176103845,/Users/maxj/Music/Rips/Spotify/AIFF/KR3TURE - ...,KR3TURE,Anthropocene [-9],7A,76,0.705268
4,116636439,/Users/maxj/Music/Rips/Spotify/AIFF/AllttA - T...,"AllttA, 20syl, Mr. J. Medeiros",The Woods [-10],9A,80,0.705254


In [89]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/HONDO - Rebeca Rebeca - 2mxyclQfXocBOTbROmNHC4.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,108284543,/Users/maxj/Music/Rips/Spotify/AIFF/Riton - Fa...,"Riton, Kah-Lo, GEE LEE",Fake ID (Coke & Rum Remix) [-10],9A,125,0.720126
1,124701476,/Users/maxj/Music/Rips/Spotify/AIFF/Weaver Bea...,Weaver Beats,Lean [-14],12A,130,0.710877
2,222229999,/Users/maxj/Music/Rips/Spotify/AIFF/David Cutt...,"David Cutter Music, Dyalla",Mangle [-9],10A,134,0.709019
3,20687581,/Users/maxj/Music/Rips/Spotify/AIFF/Axwell - I...,"Axwell, Max C, High Contrast",I Found U - High Contrast's Old Skool Revenge ...,11A,130,0.706433
4,217409767,/Users/maxj/Music/Rips/Spotify/AIFF/KAYTRANADA...,KAYTRANADA,LITE SPOTS [-11],12A,120,0.701824


In [90]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Ron Flatter - Mantequilla - Original Mix - 2obihzEzrYcoNXlouXiOZe.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,81381037,/Users/maxj/Music/Rips/Spotify/AIFF/Claptone -...,"Claptone, JAW",No Eyes - Radio Edit [-12],4A,120,0.719545
1,260413395,/Users/maxj/Music/Downloads/Soulseek/Justin Ma...,"Justin Martin, Claire George",Let's [-10],7A,121,0.709507
2,198243341,/Users/maxj/Music/Rips/Spotify/AIFF/Louie Vega...,"Louie Vega, The Martinez Brothers, Vintage Cul...",Let It Go (with Marc E. Bassy) - Vintage Cultu...,4A,124,0.706203
3,134337661,/Users/maxj/Music/Downloads/Soulseek/Dillon Fr...,"Dillon Francis, VINNE",Once Again [-8],4A,126,0.693752
4,258075238,/Users/maxj/Music/Downloads/Beatport/15808362_...,P2,Tropical Chant (Extended Mix) [-10],1A,126,0.692599


In [91]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/ZAAC - Desce Pro Play (PA PA PA) - 4zGtfOglDBfVUypabBwIjQ.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,86107411,/Users/maxj/Music/Downloads/Soulseek/AOBeats -...,"AOBeats, Mark Johns, Robokid",Peanut Butter [-9],6A,95,0.837928
1,257754158,/Users/maxj/Music/Rips/Spotify/AIFF/Cadenza - ...,"Cadenza, Ms Banks, Spice",Walk Out [-9],5A,104,0.815893
2,149354444,/Users/maxj/Music/Rips/Spotify/AIFF/Del The Fu...,Del The Funky Homosapien,Mistadobalina [-9],8A,99,0.809941
3,247796272,/Users/maxj/Music/Rips/Spotify/AIFF/Falcons - ...,"Falcons, A-1",Boss Mackin' (feat. A-1) [-9],3A,100,0.807792
4,34491553,/Users/maxj/Music/Rips/Spotify/AIFF/Falcons - ...,"Falcons, A1",Boss Mackin' [-9],3A,100,0.807792


In [92]:
sim(*get('/Users/maxj/Music/Downloads/Soulseek/AOBeats - Peanut Butter.aiff'), 10)

,id,path,artist,title,key,bpm,cosine_sim
0,34491553,/Users/maxj/Music/Rips/Spotify/AIFF/Falcons - ...,"Falcons, A1",Boss Mackin' [-9],3A,100,0.859946
1,247796272,/Users/maxj/Music/Rips/Spotify/AIFF/Falcons - ...,"Falcons, A-1",Boss Mackin' (feat. A-1) [-9],3A,100,0.859946
2,247987879,/Users/maxj/Music/Rips/Spotify/AIFF/Tropkillaz...,"Tropkillaz, Recayd Mob",Benzzz [-8],9A,94,0.846885
3,135982909,/Users/maxj/Music/Rips/Spotify/AIFF/People Und...,"People Under The Stairs, Camel MC",Acid Raindrops [-10],8A,91,0.846373
4,123301342,/Users/maxj/Music/Rips/Spotify/AIFF/ZAAC - Des...,"ZAAC, Anitta, Tyga",Desce Pro Play (PA PA PA) [-8],9A,100,0.837928
5,5681199,/Users/maxj/Music/Rips/Spotify/AIFF/Stylust - ...,"Stylust, Emotionz",Rick James (feat. Emotionz) [-7],4A,100,0.829775
6,87924734,/Users/maxj/Music/Rips/Spotify/AIFF/Danny Byrd...,Danny Byrd,Planet Earth [-8],8A,175,0.825301
7,170452298,/Users/maxj/Music/Rips/Spotify/AIFF/The Remnan...,The Remnant,Summer Dog Days [-9],1A,99,0.822349
8,101347676,/Users/maxj/Music/Rips/Spotify/AIFF/Chase & St...,"Chase & Status, Clementine Douglas",Mixed Emotions [-8],8A,175,0.818273
9,209029629,/Users/maxj/Music/Rips/Spotify/AIFF/Jean Deaux...,Jean Deaux,Recipe! [-10],4A,95,0.811172


In [93]:
sim(*get("/Users/maxj/Music/Rips/Spotify/AIFF/Air - La femme d'argent - 6tEaLXZlN8b71vWV1SSsRf.aiff"), 10)

,id,path,artist,title,key,bpm,cosine_sim
0,252909381,/Users/maxj/Music/Rips/Spotify/AIFF/Cosmic Que...,Cosmic Quest,Sunlit Echo [-8],11A,81,0.739521
1,50757859,/Users/maxj/Music/Rips/Spotify/AIFF/Redeyes - ...,"Redeyes, KinKai",Oooo Laaah [-11],4A,86,0.736692
2,81108243,/Users/maxj/Music/Rips/Spotify/AIFF/Glass Anim...,Glass Animals,Heat Waves [-6],1A,80,0.733407
3,108635005,/Users/maxj/Music/Rips/Spotify/AIFF/Netsky - E...,"Netsky, Daddy Waku, Chantal Kashala",Everybody Loves The Sunshine [-7],10A,172,0.727224
4,58739552,/Users/maxj/Music/Rips/Spotify/AIFF/Netsky - T...,"Netsky, A.CHAL",Téquila Limonada [-7],12A,170,0.711080
5,107076216,/Users/maxj/Music/Rips/Spotify/AIFF/Boombox Ca...,"Boombox Cartel, Dabow",Rock Dem - Dabow Remix [-8],7A,80,0.710651
6,14241502,/Users/maxj/Music/Rips/Spotify/AIFF/Jade Cicad...,Jade Cicada,Little Creatures [-8],9A,160,0.701531
7,144434285,/Users/maxj/Music/Rips/Spotify/AIFF/Break - La...,"Break, Celestine",Last Goodbye [-7],9A,172,0.693190
8,106879700,/Users/maxj/Music/Rips/Spotify/AIFF/Illect Rec...,"Illect Recordings, Sivion, Sareem Poems, Chang...","Memory Serves (feat. Sivion, Sareem Poems, Cha...",9A,85,0.692415
9,86843563,/Users/maxj/Music/Rips/Spotify/AIFF/Flume - De...,Flume,Depth Charge [-7],1A,79,0.682775


In [94]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Cosmic Quest - Sunlit Echo - 7hCFhxgJBPDkMurHpee6hj.aiff'), 5)

,id,path,artist,title,key,bpm,cosine_sim
0,198694821,/Users/maxj/Music/Rips/Spotify/AIFF/Whereisale...,Whereisalex,Shine [-8],11A,85,0.856157
1,144434285,/Users/maxj/Music/Rips/Spotify/AIFF/Break - La...,"Break, Celestine",Last Goodbye [-7],9A,172,0.790603
2,77666172,/Users/maxj/Music/Rips/Spotify/AIFF/VOLO - Wel...,"VOLO, Good Lee",Wellspring [-10],10A,75,0.776253
3,189195603,/Users/maxj/Music/Rips/Spotify/AIFF/Desmond Ch...,"Desmond Cheese, Roman MC",Storm's Coming (feat. Roman MC) [-10],9A,77,0.761345
4,74547264,/Users/maxj/Music/Rips/Spotify/AIFF/High Contr...,High Contrast,Twilight's Last Gleaming - Vip [-6],11A,173,0.759101


In [95]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Whereisalex - Shine - 76FR3MBARic3eh86RQjubB.aiff'), 5)

,id,path,artist,title,key,bpm,cosine_sim
0,252909381,/Users/maxj/Music/Rips/Spotify/AIFF/Cosmic Que...,Cosmic Quest,Sunlit Echo [-8],11A,81,0.856157
1,49379927,/Users/maxj/Music/Rips/Spotify/AIFF/Porter Rob...,"Porter Robinson, Amy Millan, ODESZA",Divinity - ODESZA Remix [-8],10A,90,0.809145
2,81702247,/Users/maxj/Music/Rips/Spotify/AIFF/Bassnectar...,"Bassnectar, The Glitch Mob",Paracosm [-6],11A,80,0.783973
3,74547264,/Users/maxj/Music/Rips/Spotify/AIFF/High Contr...,High Contrast,Twilight's Last Gleaming - Vip [-6],11A,173,0.781035
4,240702048,/Users/maxj/Music/Rips/Spotify/AIFF/Party Favo...,"Party Favor, DeathbyRomy",Hollow (with DeathbyRomy) [-8],12A,80,0.773885


In [97]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Bassnectar - Paracosm - 4SEVhyCF5jZnhht07sZH0P.aiff'), 6)

,id,path,artist,title,key,bpm,cosine_sim
0,240702048,/Users/maxj/Music/Rips/Spotify/AIFF/Party Favo...,"Party Favor, DeathbyRomy",Hollow (with DeathbyRomy) [-8],12A,80,0.797818
1,198694821,/Users/maxj/Music/Rips/Spotify/AIFF/Whereisale...,Whereisalex,Shine [-8],11A,85,0.783973
2,164042141,/Users/maxj/Music/Rips/Spotify/AIFF/An-Ten-Nae...,An-Ten-Nae,Raindrops On Roses feat. Alice. D [-8],11A,75,0.779599
3,151984190,/Users/maxj/Music/Rips/Spotify/AIFF/Big Wild -...,Big Wild,I Just Wanna [-8],11A,74,0.769871
4,65514448,/Users/maxj/Music/Rips/Spotify/AIFF/TNGHT - Tu...,"TNGHT, Lunice, Hudson Mohawke",Tums [-7],1A,83,0.761720
5,172406762,/Users/maxj/Music/Rips/Spotify/AIFF/Big Wild -...,Big Wild,Venice Venture [-10],12B,80,0.746685


In [99]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Party Favor - Hollow (with DeathbyRomy) - 7pZJtzfLAFB0NxWzLr3f85.aiff'), 6)

,id,path,artist,title,key,bpm,cosine_sim
0,177364745,/Users/maxj/Music/Rips/Spotify/AIFF/The Glitch...,The Glitch Mob,Antireal [-7],2A,150,0.818815
1,79576886,/Users/maxj/Music/Rips/Spotify/AIFF/GRiZ - Jui...,"GRiZ, Blunts & Blondes",Juicy [-5],7A,75,0.818705
2,81702247,/Users/maxj/Music/Rips/Spotify/AIFF/Bassnectar...,"Bassnectar, The Glitch Mob",Paracosm [-6],11A,80,0.797818
3,139363062,/Users/maxj/Music/Rips/Spotify/AIFF/The Glitch...,The Glitch Mob,E-Motion [-7],4A,162,0.787090
4,201348583,/Users/maxj/Music/Rips/Spotify/AIFF/Oski - Cal...,"Oski, NEOTEK",California [-6],6A,80,0.782687
5,198694821,/Users/maxj/Music/Rips/Spotify/AIFF/Whereisale...,Whereisalex,Shine [-8],11A,85,0.773885


In [100]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/The Glitch Mob - Higher - 3izf3TMsU9gtxEpLm4J7aO.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,256906281,/Users/maxj/Music/Rips/Spotify/AIFF/The Glitch...,The Glitch Mob,The Flavor [-7],4A,135,0.858280
1,109393904,/Users/maxj/Music/Rips/Spotify/AIFF/MKII - Hig...,"MKII, Jasmine Knight",Higher [-8],4A,140,0.817627
2,120159950,/Users/maxj/Music/Rips/Spotify/AIFF/COFRESI - ...,"COFRESI, Flamingosis",Voodoo (feat. Flamingosis) [-9],4A,69,0.811013
3,63970230,/Users/maxj/Music/Rips/Spotify/AIFF/Oshi - Sta...,Oshi,State of Flux [-6],5A,70,0.805998
4,23263369,/Users/maxj/Music/Rips/Spotify/AIFF/TroyBoi - ...,"TroyBoi, UZ",Yalla [-7],4A,70,0.794779


In [102]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/MKII - Higher - 0oSsN5AbUaHbgH1XTf9lyM.aiff'), 6)

,id,path,artist,title,key,bpm,cosine_sim
0,256906281,/Users/maxj/Music/Rips/Spotify/AIFF/The Glitch...,The Glitch Mob,The Flavor [-7],4A,135,0.834410
1,83479653,/Users/maxj/Music/Rips/Spotify/AIFF/GRiZ - A N...,"GRiZ, Matisyahu",A New Day [-8],4A,150,0.829278
2,190569724,/Users/maxj/Music/Rips/Spotify/AIFF/Sofi Tukke...,Sofi Tukker,Energia [-8],4A,135,0.824303
3,148845363,/Users/maxj/Music/Rips/Spotify/AIFF/NAYANA IZ ...,NAYANA IZ,TNT [-11],2A,72,0.820550
4,152777176,/Users/maxj/Music/Rips/Spotify/AIFF/The Glitch...,The Glitch Mob,Higher [-7],4A,137,0.817627
5,176372387,/Users/maxj/Music/Rips/Spotify/AIFF/Cardi B - ...,"Cardi B, Bad Bunny, J Balvin",I Like It [-8],4A,136,0.814676


In [103]:
sim(*get("/Users/maxj/Music/Rips/Spotify/AIFF/Kink - Valentine's Groove - 6xTSXB9aXvk3Xy4FioT7fn.aiff"))

,id,path,artist,title,key,bpm,cosine_sim
0,64984568,/Users/maxj/Music/Rips/Spotify/AIFF/Saison - M...,"Saison, Miss Yankey, Scott Diaz",Making Shapes - Scott Diaz Remix [-9],9A,124,0.799245
1,51564322,/Users/maxj/Music/Edits/Digital Afrika - Kings...,"Digital Afrika, Cazeaux Oslo",Kings for a Minute [-1st] [-10],9A,124,0.771483
2,106574024,/Users/maxj/Music/Downloads/Bandcamp/Digital A...,"Digital Afrika, Cazeaux Oslo",Kings for a Minute [-10],4A,124,0.766384
3,77883043,/Users/maxj/Music/Rips/Spotify/AIFF/Flight Fac...,"Flight Facilities, Channel Tres",Lights Up (feat. Channel Tres) - Extended [-9],5A,118,0.765835
4,82336707,/Users/maxj/Music/Rips/Spotify/AIFF/Michael Br...,"Michael Brun, UNIIQU3",How About This [-7],9A,118,0.761208


In [104]:
sim(*get("/Users/maxj/Music/Rips/Spotify/AIFF/HONDO - Rebeca Rebeca - 2mxyclQfXocBOTbROmNHC4.aiff"))

,id,path,artist,title,key,bpm,cosine_sim
0,108284543,/Users/maxj/Music/Rips/Spotify/AIFF/Riton - Fa...,"Riton, Kah-Lo, GEE LEE",Fake ID (Coke & Rum Remix) [-10],9A,125,0.720126
1,124701476,/Users/maxj/Music/Rips/Spotify/AIFF/Weaver Bea...,Weaver Beats,Lean [-14],12A,130,0.710877
2,222229999,/Users/maxj/Music/Rips/Spotify/AIFF/David Cutt...,"David Cutter Music, Dyalla",Mangle [-9],10A,134,0.709019
3,20687581,/Users/maxj/Music/Rips/Spotify/AIFF/Axwell - I...,"Axwell, Max C, High Contrast",I Found U - High Contrast's Old Skool Revenge ...,11A,130,0.706433
4,217409767,/Users/maxj/Music/Rips/Spotify/AIFF/KAYTRANADA...,KAYTRANADA,LITE SPOTS [-11],12A,120,0.701824


In [105]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Colombo - After the Storm - Original Mix - 6C0LkRnmWUJOA3Che4Ia0h.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,35141326,/Users/maxj/Music/Rips/Spotify/AIFF/Disclosure...,"Disclosure, Eko Roosevelt",Tondo [-10],5A,132,0.705527
1,77883043,/Users/maxj/Music/Rips/Spotify/AIFF/Flight Fac...,"Flight Facilities, Channel Tres",Lights Up (feat. Channel Tres) - Extended [-9],5A,118,0.678419
2,61048462,/Users/maxj/Music/Rips/Spotify/AIFF/Idris Elba...,"Idris Elba, Eliza Legzdina, Low Steppa",Fudge - Low Steppa Remix - Edit [-9],3A,126,0.666684
3,154333294,/Users/maxj/Music/Rips/Spotify/AIFF/GoldFish -...,GoldFish,Hold Tight [-9],4A,127,0.665987
4,167971687,/Users/maxj/Music/Rips/Spotify/AIFF/Self Tape ...,Self Tape,Where I Go [-8],5A,126,0.664506


In [106]:
sim(*get('/Users/maxj/Music/Rips/Spotify/AIFF/Groove Armada - My Friend - Logic1000 Remix - 3xbrGyTqrtSQpIH4mkG349.aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,231010988,/Users/maxj/Music/Downloads/Beatport/16772757_...,Billen Ted,People Ain't Dancing (feat. Kah-Lo) [-9],4A,125,0.812769
1,232136634,/Users/maxj/Music/Rips/Spotify/AIFF/dialE - AO...,dialE,AOT [-10],2A,121,0.800201
2,164018606,/Users/maxj/Music/Rips/Spotify/AIFF/Flume - Ho...,"Flume, Emma Louise, Logic1000",Hollow (feat. Emma Louise) [Logic1000 Remix] [-9],5A,117,0.791868
3,234955773,/Users/maxj/Soulseek Downloads/complete/netwer...,"Josh Wink, Lil' Louis, Chris Lake",How's Your Evening So Far (Chris Lake Extended...,4A,126,0.789042
4,189924823,/Users/maxj/Music/Downloads/Beatport/15030495_...,"Lenny Kiser, VNSSA",Cue The Rhythm (Original Mix) [-9],4A,127,0.788154


In [108]:
sim(*get('/Users/maxj/Music/Downloads/Beatport/5329444_Me Roar_(Format_B Remix).aiff'))

,id,path,artist,title,key,bpm,cosine_sim
0,51564322,/Users/maxj/Music/Edits/Digital Afrika - Kings...,"Digital Afrika, Cazeaux Oslo",Kings for a Minute [-1st] [-10],9A,124,0.826184
1,69408968,/Users/maxj/Music/Rips/Spotify/AIFF/LuSiD - Su...,LuSiD,Sultan [-8],4A,124,0.803667
2,156147771,/Users/maxj/Music/Downloads/Beatport/15421985_...,Black V Neck,Like Whoa (Extended Mix) [-9],4A,126,0.802885
3,21279011,/Users/maxj/Music/Downloads/Beatport/16287295_...,John Summit,La Danza (Extended Mix) [-7],12A,126,0.800757
4,97491341,/Users/maxj/Music/Rips/Spotify/AIFF/SWAYL - Yu...,SWAYLÓ,Yucatán Dream [-8],4A,124,0.800150
